# Authentication Establishment

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Import Data

In [ ]:
!mkdir ~/.kaggle
!echo '{"username":"abhiksark","key":"xxx"}' > ~/.kaggle/kaggle.json

In [ ]:
!pip install fastai==0.7
!pip install torchtext==0.2.3
!pip install torchvision==0.1.9

!wget http://files.fast.ai/models/weights.tgz 
!mkdir /usr/local/lib/python3.6/dist-packages/fastai/weights
!tar -xvzf weights.tgz -C /usr/local/lib/python3.6/dist-packages/fastai/

In [ ]:
!kaggle competitions download -c diabetic-retinopathy-detection -p datalab

In [ ]:
!mkdir tmp

In [ ]:
%%capture
!7z x datalab/test.zip.001
!7z x datalab/train.zip.001
!unzip datalab/trainLabels.csv.zip

In [ ]:
!mv test tmp
!mv train tmp

# Pre-Processing

In [ ]:
#imports 
import pandas as pd
import os
import random
import os
import math
import numpy as np

from PIL import Image
from skimage import io
from skimage import exposure


In [ ]:

trainLabels = pd.read_csv("trainLabels.csv")
trainLabels.head()

In [ ]:
listing = os.listdir("tmp/train") 

In [ ]:
!mkdir tmp/train/0
!mkdir tmp/train/1
!mkdir tmp/train/2
!mkdir tmp/train/3
!mkdir tmp/train/4

In [ ]:
base_dir = "tmp/train/"

for file in listing:
  try:
    fileName = os.path.splitext(file)[0]
    folder_name = trainLabels.loc[trainLabels.image==fileName, 'level'].values[0]  
    os.rename(base_dir + file, base_dir + str(folder_name) + "/" + file )
  except:
    print("error")
    pass


In [ ]:
!ls tmp/train/0 | wc -l

In [ ]:
!ls tmp/train/3 | wc -l

In [ ]:
!ls tmp/train/4 | wc -l

In [ ]:
def remove_excess_images(PATH,number):
    listing = os.listdir(PATH) 
    to_delete = random.sample(listing, number)
    for file in to_delete:
        os.remove(PATH+file)

In [ ]:
# removed excess of the zero images
DEL_DIR = "tmp/train/0/"
remove_excess_images(DEL_DIR,18000)

In [ ]:
def data_aug_val(path):
    listing_test = os.listdir(path)
    for file_name in listing_test:
            image = io.imread(path+file_name)
            v_min, v_max = np.percentile(image, (0.2, 99.8))
            better_contrast = exposure.rescale_intensity(image, in_range=(v_min, v_max))
            log_correction_image = exposure.adjust_log(image)
            sigmoid_correction_image = exposure.adjust_sigmoid(image)
            horizontal_flip = image[:, ::-1]
            #color_inversion_image_with_np = np.invert(image)
            #adjusted_gamma_image = exposure.adjust_gamma(image, gamma=0.4, gain=0.9)
            io.imsave(path+str(1)+file_name,better_contrast)
            io.imsave(path+str(2)+file_name,log_correction_image)
            io.imsave(path+str(3)+file_name,sigmoid_correction_image)

In [ ]:
data_aug_val("tmp/train/3/")
data_aug_val("tmp/train/4/")

In [ ]:
import sys
from time import sleep
from tqdm import tqdm
import PIL


def resize(DIR):
    list_test =  os.listdir(DIR) 
    j=0
    size = 512,512
    with tqdm(total=len(list_test), file=sys.stdout) as pbar:
        for i in list_test:
            name = DIR + str(i)
            j=j+1
            pbar.set_description('processed: %d' % (1 + j))
            pbar.update(1)
            try:
                im = PIL.Image.open(name)
                im.thumbnail(size, PIL.Image.ANTIALIAS)
                im.save(name, "jpeg")
            except IOError:
                print("cannot create thumbnail for '%s'" % name)

In [ ]:
resize("tmp/train/0/");resize("tmp/train/1/");resize("tmp/train/2/");resize("tmp/train/3/");resize("tmp/train/4/");

In [ ]:
!mkdir tmp/val
!mkdir tmp/val/0
!mkdir tmp/val/1
!mkdir tmp/val/2
!mkdir tmp/val/3
!mkdir tmp/val/4

In [ ]:
BASE_DIR = "tmp/train/"

list_dir =  os.listdir(BASE_DIR)

for tdir in list_dir:
  list_images = os.listdir(BASE_DIR+tdir)
  print(tdir)


In [ ]:
BASE_DIR = "tmp/train/"
NEW_DIR= "tmp/val/"

list_dir =  os.listdir(BASE_DIR)

for tdir in list_dir:
  list_images = os.listdir(BASE_DIR+tdir)
  to_val = random.sample(list_images, len(list_images)//5)
  for images in to_val:
    os.rename(BASE_DIR + str(tdir) + "/"+ images, NEW_DIR + str(tdir) + "/" + images )
    

In [ ]:
%%capture
!zip -r tmp/train{.zip,}

In [ ]:
%%capture
!zip -r tmp/val{.zip,}

In [ ]:
!ls tmp

# Uploading the ZippedFile in Google Drive 

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# 2. Save Keras Model or weights on google drive

# create on Colab directory
  
model_file = drive.CreateFile({'title' : 'train.zip'})
model_file.SetContentFile('./tmp/train.zip')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# 2. Save Keras Model or weights on google drive

# create on Colab directory
  
model_file = drive.CreateFile({'title' : 'val.zip'})
model_file.SetContentFile('./tmp/val.zip')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})
